# Getting genome assembly data using NCBI Datasets command line tools

The objective of this Notebook is to demonstrate how to use NCBI Datasets command line tools to explore and download genome assembly sequence and metadata. 

## Getting started 
First, we'll download and grant execute permissions for the datasets command line tools. 
Datasets has two command line tools 
- The **datasets** tool is used to query and download sequence, annotation and metadata for all domains of life.
- The **dataformat** tool is used to convert metadata downloaded from NCBI Datasets from JSON lines format to other formats.

In [1]:
!curl --silent --remote-name 'https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/LATEST/linux-amd64/datasets'
!chmod +x datasets
!printf "downloaded datasets v%s" $(./datasets version)

downloaded datasets v11.6.0

In [1]:
!curl --silent --remote-name 'https://ftp.ncbi.nlm.nih.gov/pub/datasets/command-line/LATEST/linux-amd64/dataformat'
!chmod +x dataformat
!printf "downloaded dataformat v%s" $(./dataformat version)

downloaded dataformat v11.6.0

We'll also download the command line tool [jq](https://stedolan.github.io/jq/) to parse the datasets JSON Lines data reports into a readable format.

In [1]:
!curl --silent --location --output jq 'https://github.com/stedolan/jq/releases/download/jq-1.6/jq-linux64'
!chmod +x jq
!printf "downloaded %s" $(./jq --version)

downloaded jq-1.6

## Getting help
To get help in using the tools or any sub-commands specify --help after the command:

In [1]:
!./datasets --help

datasets is a command-line tool that is used to query and download biological sequence data
across all domains of life from NCBI databases.

Refer to NCBI's [command line start](https://www.ncbi.nlm.nih.gov/datasets/docs/command-line-start) documentation for information about getting started with the command-line tools.

Usage
  datasets [command]

Data Retrieval Commands
  summary              print a summary of a gene or genome dataset
  download             download a gene, genome or coronavirus dataset as a zip file
  rehydrate            rehydrate a downloaded, dehydrated dataset

Miscellaneous Commands
  completion           generate autocompletion scripts
  version              print the version of this client and exit
  help                 Help about any command

Flags
  -h, --help   help for datasets

Use datasets help <command> for detailed help about a command.


## Getting genome metadata

To begin, we'll use the Datasets summary genome command to explore all the available RefSeq genomes for a group of organisms.

Genome summaries can be accessed in four ways:

- accession: an NCBI Assembly accession
- organism: an organism or a taxonomical group name
- taxid: using an NCBI Taxonomy identifier, at any level.
- BioProject: using an NCBI BioProject accession

In this example, we'll view metadata for all Crustacea genome assemblies using taxon name. Additionally, we'll limit our search to genome annotated by NCBI's RefSeq group using the --refseq flag. To make the JSON output easy to read we'll use the command line parser jq. 

In [1]:
!./datasets summary genome taxon Crustacea --refseq | ./jq .

{
  "assemblies": [
    {
      "assembly": {
        "annotation_metadata": {
          "file": [
            {
              "estimated_size": "8160265",
              "type": "GENOME_GFF"
            },
            {
              "estimated_size": "60912986",
              "type": "GENOME_GBFF"
            },
            {
              "estimated_size": "15723551",
              "type": "RNA_FASTA"
            },
            {
              "estimated_size": "5579866",
              "type": "PROT_FASTA"
            },
            {
              "estimated_size": "6762708",
              "type": "GENOME_GTF"
            }
          ],
          "name": "NCBI Annotation Release 100",
          "release_date": "Mar 16, 2020",
          "release_number": "100",
          "report_url": "https://www.ncbi.nlm.nih.gov/genome/annotation_euk/Daphnia_magna/100/",
          "source": "NCBI"
        },
        "assembly_accession": "GCF_003990815.1",
        "assembly_category": "representati

If you just want to get the count of available RefSeq (GCF) genomes that fall under a particular tax name, use the --refseq flag and set --limit to NONE:

In [1]:
!./datasets summary genome taxon crustacea --refseq --limit NONE

{"total_count":6}


## Downloading genome assembly sequence and metadata 
In this section, we'll show you how to download a genome data package for one of the Crustacean genomes using the datasets download genome command. Genome data packages can be retrieved in four ways 

- accession: an NCBI Assembly accession
- organism: an organism or a taxonomical group name
- taxid: using an NCBI Taxonomy identifier, at any level.
- BioProject: using an NCBI BioProject accession

The default genome data package includes the following data (when available):

- genomic sequence (genomic.fna)
- transcript sequences (rna.fna)
- protein sequences (protein.faa)
- annotation in gff3 format (genomic.gff)
- a data report containing genome assembly and annotation metadata (assembly_data_report.jsonl)
- a sequence report listing the nucleotide sequences that comprise the genome assembly (sequence_report.jsonl)

In this example, we'll download the Datasets genome package for the <em>Penaeus vannamei</em> reference genome. For the purposes of this demonstration, we will redirect all messages from the datasets command to datasets.log.

In [1]:
!./datasets download genome taxon "penaeus vannamei" --filename pacific_white_shrimp.zip >datasets.log 2>&1
!printf "Downloaded:\n%s" "$(du -ah pacific_white_shrimp.zip)"

Downloaded:
901M	pacific_white_shrimp.zip

## Converting the Datasets assembly data report to tabular format
The Datasets genome assembly data report can be converted to tabular format using the dataformat tool. In this step, we'll use the help command to view the data fields available for conversion  

In [1]:
!./dataformat tsv genome --help


Convert Genome Assembly Data Report into TSV format.

Refer to NCBI's [command line start](https://www.ncbi.nlm.nih.gov/datasets/docs/command-line-start) documentation for information about getting started with the command-line tools.

Usage
  dataformat tsv genome [flags]

Examples
  dataformat tsv genome --inputfile human/ncbi_dataset/data/assembly_data_report.jsonl
  dataformat tsv genome --package human.zip

Flags
      --fields strings     comma-separated list of fields
                               - annot-name
                               - annot-release-date
                               - annot-report-url
                               - annot-source
                               - assm-description
                               - assm-level
                               - assm-name
                               - assm-submitter
                               - assm-type
                               - biosample-accession
                               - breed
       

Let's look at the catalog inside the package, converting this JSON into an easy-to-read table.

In [1]:
!./dataformat catalog --package pacific_white_shrimp.zip | ./jq -r '.assemblies[] | .files[] | [.filePath, .fileType] | @csv'

matched file: ncbi_dataset/data/dataset_catalog.json
GCA_003730335.1/GCA_003730335.1_ASM373033v1_genomic.fna  GENOMIC_NUCLEOTIDE_FASTA
GCA_003730335.1/sequence_report.jsonl                    SEQUENCE_REPORT
GCA_003789085.1/GCA_003789085.1_ASM378908v1_genomic.fna  GENOMIC_NUCLEOTIDE_FASTA
GCA_003789085.1/genomic.gff                              GFF3
GCA_003789085.1/protein.faa                              PROTEIN_FASTA
GCA_003789085.1/sequence_report.jsonl                    SEQUENCE_REPORT
GCF_003789085.1/GCF_003789085.1_ASM378908v1_genomic.fna  GENOMIC_NUCLEOTIDE_FASTA
GCF_003789085.1/genomic.gff                              GFF3
GCF_003789085.1/protein.faa                              PROTEIN_FASTA
GCF_003789085.1/rna.fna                                  RNA_NUCLEOTIDE_FASTA
GCF_003789085.1/sequence_report.jsonl                    SEQUENCE_REPORT
assembly_data_report.jsonl                               DATA_REPORT


Now we'll use the dataformat tool to convert a default set of data fields into tsv format.

In [1]:
!./dataformat tsv genome --package pacific_white_shrimp.zip --fields assm-name,refseq-assm-accession,genbank-assm-accession,refseq-category,number-of-contigs,number-of-scaffolds

matched file: ncbi_dataset/data/assembly_data_report.jsonl
Assembly name  RefSeq assembly accession  GenBank assembly accession  Refseq category        Number of contigs  Number of scaffolds
ASM373033v1    na                         GCA_003730335.1             na                     19584              19584
ASM378908v1    GCF_003789085.1            GCA_003789085.1             representative genome  33019              4682
ASM378908v1    GCF_003789085.1            GCA_003789085.1             representative genome  33019              4682


Next, we can list the first 30 FASTA deflines for one of the assemblies

In [1]:
!unzip -q -c pacific_white_shrimp.zip ncbi_dataset/data/GCA_003730335.1/GCA_003730335.1_ASM373033v1_genomic.fna | grep '^>' | head --lines 30

>QWLK01000001.1 Penaeus vannamei breed F1 c13686_f1p0_850, whole genome shotgun sequence
>QWLK01000002.1 Penaeus vannamei breed F1 c11163_f1p0_1159, whole genome shotgun sequence
>QWLK01000003.1 Penaeus vannamei breed F1 c17502_f1p0_1835, whole genome shotgun sequence
>QWLK01000004.1 Penaeus vannamei breed F1 c7653_f1p0_831, whole genome shotgun sequence
>QWLK01000005.1 Penaeus vannamei breed F1 c9813_f1p1_862, whole genome shotgun sequence
>QWLK01000006.1 Penaeus vannamei breed F1 c10679_f1p0_1560, whole genome shotgun sequence
>QWLK01000007.1 Penaeus vannamei breed F1 c4860_f1p0_4016, whole genome shotgun sequence
>QWLK01000008.1 Penaeus vannamei breed F1 c5989_f1p0_2744, whole genome shotgun sequence
>QWLK01000009.1 Penaeus vannamei breed F1 c9240_f6p4_3285, whole genome shotgun sequence
>QWLK01000010.1 Penaeus vannamei breed F1 c4403_f1p0_1462, whole genome shotgun sequence
>QWLK01000011.1 Penaeus vannamei breed F1 c11798_f4p3_3667, whole genome shotgun sequence
>QWLK01000012.1 Pen